# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [1]:
#pip install --upgrade langchain

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
llm_model = "gpt-3.5-turbo"

In [3]:
from langchain.chains import RetrievalQA
from langchain_openai.chat_models.base import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.llms import OpenAI
import pandas as pd

C:\Users\rauna\Anaconda3\envs\kaggle\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (None)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
file = 'netflix_small.csv'
loader = CSVLoader(file_path=file)

In [5]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings

In [6]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [7]:
index = VectorstoreIndexCreator(
    embedding=embeddings,
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

C:\Users\rauna\Anaconda3\envs\kaggle\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [48]:
llm = ChatOpenAI(temperature=1, model=llm_model)

In [50]:
query ="Give me a three word description for kota factory"

In [51]:
response = index.query(query, llm = llm)

In [52]:
display(Markdown(response))

Campus life navigation.

## Step By Step

In [53]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [54]:
docs = loader.load()

In [65]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [66]:
print(len(embed))

1536


In [67]:
print(embed[:5])

[0.005683124531060457, -0.005880772136151791, -0.06271202117204666, 0.022671684622764587, -0.05137617886066437]


In [69]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [70]:
query = "Please suggest a tv show about students"

In [71]:
docs = db.similarity_search(query)

In [72]:
len(docs)

4

In [76]:
for d in docs:
    print(d.page_content,'\n\n')

title: Dear White People
description: Students of color navigate the daily slights and slippery politics of life at an Ivy League college that's not nearly as "post-racial" as it thinks. 


title: Kota Factory
description: In a city of coaching centers known to train Indiaâ€™s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life. 


title: Confessions of an Invisible Girl
description: When the clever but socially-awkward TetÃª joins a new school, she'll do anything to fit in. But the queen bee among her classmates has other ideas. 


title: Blood & Water
description: After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth. 




In [77]:
retriever = db.as_retriever()

In [80]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [82]:
print(qdocs)

title: Dear White People
description: Students of color navigate the daily slights and slippery politics of life at an Ivy League college that's not nearly as "post-racial" as it thinks.title: Kota Factory
description: In a city of coaching centers known to train Indiaâ€™s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life.title: Confessions of an Invisible Girl
description: When the clever but socially-awkward TetÃª joins a new school, she'll do anything to fit in. But the queen bee among her classmates has other ideas.title: Blood & Water
description: After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.


In [88]:
response = llm.call_as_llm(f"{qdocs} Question: Please tell in 5 words how  each tv show is about the studens in  markdown table") 


In [89]:
display(Markdown(response))

| Title                     | Description                                    |
|---------------------------|-----------------------------------------------|
| Dear White People         | Students navigating racial dynamics          |
| Kota Factory              | Student life in coaching center              |
| Confessions of an Invisible Girl | Socially-awkward student fitting in |
| Blood & Water             | Teen searching for abducted sister          |

In [92]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [93]:
query =  "Please tell in 5 words how  each tv show is about the studens in  markdown table"

In [94]:
response = qa_stuff.run(query)

C:\Users\rauna\Anaconda3\envs\kaggle\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


In [95]:
display(Markdown(response))

| TV Show                   | Description                             |
|---------------------------|-----------------------------------------|
| Dear White People         | College students facing racial issues  |
| Kota Factory              | Students navigating campus life        |
| Confessions of an Invisible Girl | Awkward student wanting acceptance |
| My Little Pony: A New Generation | Unity among different pony species |

In [96]:
response = index.query(query, llm=llm)

In [97]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

In [99]:
print(response)

| TV Show                       | 5-word Description                        |
|------------------------------|------------------------------------------|
| Dear White People               | Navigating college as students              |
| Kota Factory                   | Student life in coaching city              |
| Confessions of an Invisible Girl | Social struggles in new school           |
| My Little Pony: A New Generation | Unity among Equestria's ponies           |


Reminder: Download your notebook to you local computer to save your work.